1. 平移不变性
2. 局部性

对全连接层使用平移不变性和局部性得到卷积层

1. 卷积层将输入 和 核矩阵进行交叉相关，加上偏移后得到输出
2. 核矩阵 和 偏移 是可学习的参数
3. 核矩阵的大小是超参数

In [2]:
import torch
from torch import nn
from d2l import torch as d2l

In [11]:
def corr2d(X, K):
    h, w = K.shape
    Y = torch.zeros((X.shape[0] - h + 1, X.shape[1] - w + 1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i, j] = (X[i:i+h, j:j+w] * K).sum()
    return Y


In [12]:
a = torch.tensor([[3,2],[1,2]])
b = torch.tensor([[1,2],[4,2]])

(a*b).sum(),(a*b)

(tensor(15),
 tensor([[3, 4],
         [4, 4]]))

In [13]:
X = torch.tensor([[0, 1, 2], [3, 4, 5], [6, 7, 8]])
K = torch.tensor([[0, 1], [2, 3]])
corr2d(X, K)

tensor([[19., 25.],
        [37., 43.]])

卷积层

In [ ]:
class Conv2D(nn.Module):
    def __init__(self, kernel_size):
        super().__init__()
        self.weight = nn.Parameter(torch.rand(kernel_size))
        self.bias = nn.Parameter(torch.zeros(1))

    def forward(self, X):
        return corr2d(X, self.weight) + self.bias

tensor([0.])